In [1]:
!pip install bs4

In [2]:
# import requests
# from bs4 import BeautifulSoup

# def search_player(player_name):
#     base_url = "https://fbref.com"
#     search_url = f"{base_url}/en/search/search.fcgi?&search={player_name.replace(' ', '-')}"

#     # Send a GET request to the search URL
#     response = requests.get(search_url)
#     response.raise_for_status()
#     # print(response.json)
    

#     # # Create a BeautifulSoup object to parse the HTML content
#     soup = BeautifulSoup(response.content, 'html.parser')

#     res = soup.find_all('div',class_='search-item-name')
    
#      # Find all the search result items
#     search_results = soup.find_all('div', class_='search-item-name')

#     # Store the player names and URLs in a dictionary
#     players = {}
#     for result in search_results:
#         player_link = result.find('a')['href']
#         player_url = f"{base_url}{player_link}"
#         player_name = result.text.strip()
#         players[player_name] = player_url

#     for key,val in players.items():
#         print(f"PlayerName :{key} \n URL:{val}\n")

#     # return players


# # Example usage
# player_name = "Mason Mount"
# player_url = search_player(player_name)



In [3]:
# import requests
# from bs4 import BeautifulSoup

# def search_player(player_name):
#     base_url = "https://fbref.com"
#     search_url = f"{base_url}/en/search/search.fcgi?search={player_name.replace(' ', '+')}"

#     # Send a GET request to the search URL
#     response = requests.get(search_url)
#     response.raise_for_status()

#     # Create a BeautifulSoup object to parse the HTML content
#     soup = BeautifulSoup(response.content, 'html.parser')

#     # Check if there is a redirect
#     if response.url.startswith(f"{base_url}/en/players"):
#         return response.url

#     # Find all the search result items
#     search_results = soup.find_all('div', class_='search-item-name')

#     # Store the player names and URLs in a dictionary
#     players = {}
#     for result in search_results:
#         player_link = result.find('a')['href']
#         player_url = f"{base_url}{player_link}"
#         player_name = result.find('a').text.strip()
#         players[player_name] = player_url

#     return players

# # Example usage
# player_name = "Messi"
# player_info = search_player(player_name)

# if isinstance(player_info, dict):
#     for name, url in player_info.items():
#         print(f"{name}\n {url}\n")
# elif isinstance(player_info, str):
#     print(f"Player URL: {player_info}")
# else:
#     print("Player not found.")


In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
scoutURL = 'https://fbref.com/en/players/d70ce98e/scout/365_m1/Lionel-Messi-Scouting-Report'

# scoutURL = 'https://fbref.com/en/players/507c7bdf/scout/365_m1/Bruno-Fernandes-Scouting-Report'

# scoutURL = 'https://fbref.com/en/players/f70e83a1/scout/365_m1/Bruno-Hyanem-Scouting-Report'
response = requests.get(scoutURL)
response.raise_for_status()

# Create a BeautifulSoup object to parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
div = soup.find('div',id='all_scout_summary')
# div = soup.find('div',id='all_scout_full')



In [7]:
tables = div.find_all('table')
tableData = []
for table in tables:
    tbody = table.find('tbody')

    # TODO: the header will have the what type for that table
    # eg: {'vs forwards':[] , 'vs midfielder':[]} 
    headerType = {}

    # Extract the data from tbody and store it in a list
    tbody_data = []
    rows = tbody.find_all('tr')
    for row in rows:
        row_data = []
        title = row.find('th')
        titleValue = title.get_text(strip=True)
        row_data.append(titleValue)


        # Extract the text from each cell in the row and append it to the tbody_data list
        cells = row.find_all('td')

        if all(cell.get_text(strip=True) == '' for cell in cells):
            continue  # Skip the empty row

        
        for cell in cells:
            text = cell.get_text(strip=True)
            if text.endswith('%'):
                row_data.append(text[:-1])
            else:
                row_data.append(text) 
        tbody_data.append(row_data)
    
    # Append the tbody_data list to the table_body_data list
    tableData.append(tbody_data)

print(tableData[0])

[['Non-Penalty Goals', '0.52', '82'], ['Non-Penalty xG', '0.47', '85'], ['Shots Total', '3.97', '96'], ['Assists', '0.52', '99'], ['xAG', '0.41', '99'], ['npxG + xAG', '0.88', '97'], ['Shot-Creating Actions', '6.23', '99'], ['Passes Attempted', '64.25', '99'], ['Pass Completion %', '80.9', '95'], ['Progressive Passes', '9.21', '99'], ['Progressive Carries', '3.63', '97'], ['Successful Take-Ons', '3.03', '99'], ['Touches (Att Pen)', '4.70', '44'], ['Progressive Passes Rec', '6.47', '60'], ['Tackles', '0.78', '69'], ['Interceptions', '0.13', '35'], ['Blocks', '0.37', '13'], ['Clearances', '0.00', '2'], ['Aerials won', '0.05', '1']]


In [8]:
# calculates the average percentile for each category in the complete scouting report 
def averageStatForFullScouting(STAT):
    for stat in STAT:
        total = 0
        for eachval in stat:
            total+=int(eachval[-1])
    return total//len(stat) 
    

Complete Scouting Report has 135 Stats.
Summary Scouting Report has 19 Stats



In [9]:
standardStats = [table[:15] for table in tableData]
shootingStats = [table[15:30] for table in tableData]
passingStats = [table[30:52] for table in tableData]
passTypes = [table[52:67] for table in tableData]
goalShotCreation=[table[67:81] for table in tableData]
defense=[table[81:97] for table in tableData]
posession = [table[97:119] for table in tableData]
miscStats = [table[119:]for table in tableData]
    

In [10]:
labels = []
result = []
def getPolarVals():
    for table in tableData:
        value = []
        # print(table[0])
        for item in table:
            if item[0] not in labels:
                labels.append(item[0])
            value.append(int(item[-1]))
        result.append(value)

getPolarVals()
print(labels)
print(result)

['Non-Penalty Goals', 'Non-Penalty xG', 'Shots Total', 'Assists', 'xAG', 'npxG + xAG', 'Shot-Creating Actions', 'Passes Attempted', 'Pass Completion %', 'Progressive Passes', 'Progressive Carries', 'Successful Take-Ons', 'Touches (Att Pen)', 'Progressive Passes Rec', 'Tackles', 'Interceptions', 'Blocks', 'Clearances', 'Aerials won']
[[82, 85, 96, 99, 99, 97, 99, 99, 95, 99, 97, 99, 44, 60, 69, 35, 13, 2, 1], [96, 98, 99, 99, 98, 99, 99, 99, 87, 99, 68, 95, 73, 30, 14, 3, 3, 1, 3]]


In [11]:

import matplotlib.pyplot as plt
import numpy as np

# print(labels)
labelPlacement = np.linspace(start=0,stop=2*np.pi,num = len(labels))
# labelPlacement = np.concatenate((labelPlacement,[labelPlacement[0]]))
# result[0].append(result[0][0])
# # radians
# print('radians', labelPlacement)
# # degrees
# print('degrees', np.degrees(labelPlacement))



In [12]:
# plt.figure(figsize=(10,10))
# plt.subplot(polar=True)
# i=0
# for x in result:
#     plt.plot(labelPlacement,x,label=f"{i}")
# lines, labels = plt.thetagrids(np.degrees(labelPlacement), labels=labels)
# plt.legend


In [17]:
print(result[0])
print(labels)

[82, 85, 96, 99, 99, 97, 99, 99, 95, 99, 97, 99, 44, 60, 69, 35, 13, 2, 1]
['Non-Penalty Goals', 'Non-Penalty xG', 'Shots Total', 'Assists', 'xAG', 'npxG + xAG', 'Shot-Creating Actions', 'Passes Attempted', 'Pass Completion %', 'Progressive Passes', 'Progressive Carries', 'Successful Take-Ons', 'Touches (Att Pen)', 'Progressive Passes Rec', 'Tackles', 'Interceptions', 'Blocks', 'Clearances', 'Aerials won']


In [13]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
# labels=[*labels,labels[0]]

# result[0] = [*result[0],result[0][0]]

fig = go.Figure(
    data=go.Scatterpolar(r=result[0],theta=labels, fill='toself'),
   
    # layout=go.Layout(
    #     title=go.layout.Title(text='Test'),
    #     polar={'radialaxis': {'visible': True}},
    #     showlegend=True
    # )
)

# df = pd.DataFrame(dict(
#     r=result[0],
#     theta= labels
# ))

# fig = px.line_polar(df,r='r',theta='theta',line_close=True)
# fig.update_traces(fill='toself')
# fig.update_layout(
#   polar=dict(
#     radialaxis=dict(
#       visible=True
#     ),
#   ),
#   showlegend=False
# )
fig.show()

In [14]:
print(result[0])

[82, 85, 96, 99, 99, 97, 99, 99, 95, 99, 97, 99, 44, 60, 69, 35, 13, 2, 1]


In [15]:
# print(averageStatForFullScouting(passingStats))

For Summary Scouting no need for the breakdown, can just return the table

In [16]:
import os
# if os.path.exists("results"):
# os.mkdir("results/")
# os.mkdir("results/test")

# fig.write_image("results/test/fig.png")
# fig.write_image("results/fig.png")
